In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_dengue = pd.read_csv('../data/clean/dengue_cases_by_district.csv', index_col=0)
df_dengue.index = pd.to_datetime(df_dengue.index)

# Combine Talomo districts
df_dengue['TALOMO'] = df_dengue['TALOMO'] + df_dengue['TALOMO NORTH']
df_dengue.drop(columns=['TALOMO NORTH'], inplace=True)

df_dengue

,AGDAO,BAGUIO,BUHANGIN,BUNAWAN,CALINAN,DISTRICT A,DISTRICT B,DISTRICT C,DISTRICT D,MARILOG,PAQUIBATO,TALOMO,TORIL,TUGBOK,TOTAL
index,,,,,,,,,,,,,,,
2009-01-04,8.0,2.0,16.0,6.0,2.0,6.0,1.0,9.0,0.0,0.0,1.0,12.0,4.0,0.0,67.0
2009-01-11,4.0,0.0,10.0,7.0,1.0,2.0,2.0,5.0,1.0,1.0,0.0,16.0,0.0,1.0,50.0
2009-01-18,3.0,0.0,9.0,3.0,3.0,5.0,2.0,2.0,2.0,0.0,0.0,11.0,3.0,3.0,46.0
2009-01-25,3.0,1.0,22.0,3.0,2.0,8.0,3.0,3.0,3.0,1.0,0.0,15.0,3.0,0.0,67.0
2009-02-01,8.0,0.0,17.0,6.0,0.0,3.0,5.0,1.0,1.0,0.0,0.0,15.0,3.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-26,8.0,0.0,15.0,4.0,0.0,8.0,7.0,7.0,2.0,0.0,0.0,22.0,5.0,2.0,80.0
2017-12-03,3.0,1.0,26.0,5.0,1.0,3.0,1.0,1.0,1.0,0.0,1.0,24.0,5.0,4.0,76.0
2017-12-10,5.0,0.0,26.0,9.0,0.0,1.0,3.0,6.0,5.0,1.0,1.0,28.0,4.0,2.0,91.0


In [3]:
df_dengue.append(df_dengue.sum(numeric_only=True), ignore_index=True).iloc[-1]

AGDAO          2286.0
BAGUIO          108.0
BUHANGIN       7353.0
BUNAWAN        2124.0
CALINAN         603.0
DISTRICT A     1611.0
DISTRICT B     1737.0
DISTRICT C     1476.0
DISTRICT D     1071.0
MARILOG         126.0
PAQUIBATO       495.0
TALOMO         7542.0
TORIL          1152.0
TUGBOK          990.0
TOTAL         28674.0
Name: 468, dtype: float64

In [4]:
import geopandas as gpd

df_districts = gpd.read_file('../data/raw/davao_davaodelsur/davao_davaodelsur.shp')
# df_districts.dropna(subset=['bngy'], inplace=True)
df_districts = df_districts[['bngy', 'geometry']]
df_districts

,bngy,geometry
0,Paradise Embak,"POLYGON ((125.51120 7.34652, 125.51125 7.33979..."
1,Saloy,"POLYGON ((125.42353 7.29139, 125.42389 7.28782..."
2,Buda,"POLYGON ((125.36275 7.53470, 125.34100 7.53794..."
3,Gumitan,"POLYGON ((125.41888 7.56718, 125.42005 7.56001..."
4,Datu Salumay,"POLYGON ((125.36275 7.53470, 125.34482 7.52088..."
...,...,...
170,Daliao,"POLYGON ((125.51804 7.02010, 125.51772 7.01976..."
171,Atan-Awe,"POLYGON ((125.40971 7.01090, 125.40945 7.00766..."
172,Sibulan,"POLYGON ((125.40899 6.98560, 125.40592 6.97817..."
173,Baracatan,"POLYGON ((125.43409 6.98652, 125.43453 6.98416..."


In [5]:
davao_bngy_dict = {
    'TALOMO': ['Talomo (Pob.)', 'Bago Aplaya', 'Bago Gallera', 'Baliok', 'Bucana', 'Catalunan Grande', 'Catalunan Pequeño', 'Dumoy',
               'Langub', 'Ma-a', 'Magtuod', 'Matina Aplaya', 'Matina Crossing', 'Matina Pangi', 'None'],
    'AGDAO': ['Agdao', 'Centro (San Juan)', 'Gov. Vicente Duterte', 'Gov. Paciano Bangoy', 'Kap. Tomas Monteverde, Sr.', 'Lapu-lapu',
             'Leon Garcia, Sr.', 'Rafael Castillo', 'San Antonio', 'Ubalde', 'Wilfredo Aquino'],
    'BUHANGIN': ['Buhangin (Pob.)', 'Acacia', 'Pampanga', 'Cabantian', 'Callawa', 'Communal', 'Indangan', 'Mandug', 'Sasa',
                'Tigatto', 'Waan'],
    'BUNAWAN': ['Bunawan (Pob.)', 'Alejandra Navarro (Lasang)', 'Gatungan', 'Mahayag', 'Mudiang', 'Panacan', 'San Isidro (Licanan)',
               'Tibungco'],
    'PAQUIBATO': ['Paquibato (Pob.)', 'Colosas', 'Fatima (Benowang)', 'Lumiad', 'Mabuhay', 'Malabog', 'Mapula', 'Panalum', 'Pandaitan',
                 'Paradise Embak', 'Salapawan', 'Sumimao', 'Tapak'],
    'BAGUIO': ['Baguio (Pob.)', 'Cadalian', 'Carmen', 'Gumalang', 'Malagos', 'Tambobong', 'Tawan-tawan', 'Wines'],
    'CALINAN': ['Calinan (Pob.)', 'Biao Joaquin', 'Cawayan', 'Dacudao', 'Dalagdag', 
               'Dominga', 'Inayangan', 'Lacson', 'Lamanan', 'Lampianao', 'Megkawayan', 'Pangyan', 'Riverside', 'Saloy', 'Sirib',
               'Subasta', 'Talomo River', 'Tamayong', 'Wangan'],
    'MARILOG': ['Baganihan', 'Bantol', 'Buda', 'Dalag', 'Datu Salumay', 'Gumitan', 'Magsaysay', 'Malamba', 'Marilog', 'Salaysay', 'Suawan (Tuli)',
               'Tamugan'],
    'TORIL': ['Toril (Pob.)', 'Alambre', 'Atan-Awe', 'Bangkas Heights', 'Baracatan', 'Bato', 'Bayabas', 'Camansi', 'Catigan', 'Crossing Bayabas',
             'Daliaon Plantation', 'Daliao', 'Eden', 'Kilate', 'Lizada', 'Lubogan', 'Marapangi', 'Mulig',
             'Sibulan', 'Sirawan', 'Tagluno', 'Tagurano', 'Tibuloy', 'Tungakalan'],
    'TUGBOK': ['Tugbok (Pob.)', 'Angalan', 'Bago Oshiro', 'Balengaeng', 'Biao Escuela', 'Biao Guianga', 'Los Amigos', 'Manambulan',
              'Manuel Guianga', 'Matina Biao', 'Mintal', 'New Carmen', 'New Valencia', 'Santo Niño', 'Tacunan', 'Tagakpan', 'Talandang',
              'Ula']
}

In [6]:
def map_nonpob_districts(bngy):
    
    for district in davao_bngy_dict:
        if bngy in davao_bngy_dict.get(district):
            return district
        
    return None

In [7]:
district_shapes = gpd.GeoDataFrame()
df_districts['district'] = df_districts['bngy'].apply(lambda d: map_nonpob_districts(d))
df_districts.loc[19, 'district'] = 'TALOMO'
df_nonpob_districts = df_districts.dropna(subset=['district'])
district_shapes = district_shapes.append(df_nonpob_districts.dissolve(by='district'))
district_shapes

,geometry,bngy
district,,
AGDAO,"POLYGON ((125.62180 7.06693, 125.62167 7.06677...",Rafael Castillo
BAGUIO,"POLYGON ((125.37299 7.13014, 125.37081 7.13009...",Gumalang
BUHANGIN,"MULTIPOLYGON (((125.62729 7.10552, 125.62293 7...",Acacia
BUNAWAN,"POLYGON ((125.65847 7.16514, 125.65858 7.16472...",Alejandra Navarro (Lasang)
CALINAN,"POLYGON ((125.39021 7.10179, 125.38958 7.09528...",Saloy
MARILOG,"POLYGON ((125.40245 7.23848, 125.40678 7.23933...",Buda
PAQUIBATO,"POLYGON ((125.51071 7.26952, 125.50899 7.24984...",Paradise Embak
TALOMO,"MULTIPOLYGON (((125.48836 6.98291, 125.48828 6...",Langub
TORIL,"POLYGON ((125.40584 6.96550, 125.38612 6.96714...",Tagurano


In [8]:

df_districts.loc[19, :]

bngy                                                     None
geometry    (POLYGON ((125.7091668474495 7.298739529745008...
district                                               TALOMO
Name: 19, dtype: object

In [9]:
# function to map poblacion districts to match with df_dengue
def map_district_poblacion(district):
    if '-A' in district:
        return 'DISTRICT A'
    elif '-B' in district:
        return 'DISTRICT B'
    elif '-C' in district:
        return 'DISTRICT C'
    elif '-D' in district:
        return 'DISTRICT D'
    else:
        return 'ERROR'

In [10]:
# get districts matching poblacion format
districts_pob = df_districts[(df_districts.bngy.str.contains('-')) & (df_districts.bngy.str.contains('(Pob.)'))][['bngy', 'geometry']]
# map poblacion districts
districts_pob['bngy'] = districts_pob['bngy'].apply(lambda s: map_district_poblacion(s))
district_shapes = district_shapes.append(districts_pob.dissolve(by='bngy'))

c:\users\eric\scoop\apps\miniconda3\current\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [11]:
district_shapes = district_shapes[['geometry']]
district_shapes

,geometry
AGDAO,"POLYGON ((125.62180 7.06693, 125.62167 7.06677..."
BAGUIO,"POLYGON ((125.37299 7.13014, 125.37081 7.13009..."
BUHANGIN,"MULTIPOLYGON (((125.62729 7.10552, 125.62293 7..."
BUNAWAN,"POLYGON ((125.65847 7.16514, 125.65858 7.16472..."
CALINAN,"POLYGON ((125.39021 7.10179, 125.38958 7.09528..."
MARILOG,"POLYGON ((125.40245 7.23848, 125.40678 7.23933..."
PAQUIBATO,"POLYGON ((125.51071 7.26952, 125.50899 7.24984..."
TALOMO,"MULTIPOLYGON (((125.48836 6.98291, 125.48828 6..."
TORIL,"POLYGON ((125.40584 6.96550, 125.38612 6.96714..."
TUGBOK,"POLYGON ((125.52126 7.08325, 125.51419 7.07111..."


In [12]:
district_population = {
    'POBLACION': 174121,
    'TALOMO': 418615,
    'AGDAO': 102267,
    'BUHANGIN': 293118,
    'BUNAWAN': 152102,
    'PAQUIBATO': 44763,
    'BAGUIO': 33873,
    'CALINAN': 92075,
    'MARILOG': 52201,
    'TORIL': 148522,
    'TUGBOK': 121334,
}

def map_population(district):
    if 'DISTRICT' in district:
        return district_population.get('POBLACION')/4
    else:
        return district_population.get(district)

In [13]:
district_shapes['population'] = district_shapes.index.map(lambda s: map_population(s))
district_shapes

,geometry,population
AGDAO,"POLYGON ((125.62180 7.06693, 125.62167 7.06677...",102267.00
BAGUIO,"POLYGON ((125.37299 7.13014, 125.37081 7.13009...",33873.00
BUHANGIN,"MULTIPOLYGON (((125.62729 7.10552, 125.62293 7...",293118.00
BUNAWAN,"POLYGON ((125.65847 7.16514, 125.65858 7.16472...",152102.00
CALINAN,"POLYGON ((125.39021 7.10179, 125.38958 7.09528...",92075.00
MARILOG,"POLYGON ((125.40245 7.23848, 125.40678 7.23933...",52201.00
PAQUIBATO,"POLYGON ((125.51071 7.26952, 125.50899 7.24984...",44763.00
TALOMO,"MULTIPOLYGON (((125.48836 6.98291, 125.48828 6...",418615.00
TORIL,"POLYGON ((125.40584 6.96550, 125.38612 6.96714...",148522.00
TUGBOK,"POLYGON ((125.52126 7.08325, 125.51419 7.07111...",121334.00


In [14]:
district_shapes.to_file("../data/clean/davao_districts.shp")